In [1]:
from __future__ import division
from sys import exit
from math import sqrt
from numpy import array
from scipy.optimize import fmin_l_bfgs_b
from scipy.optimize import basinhopping
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
import os

def MAPE(params, *kwargs):
	Y = kwargs[0]
	type = kwargs[1]
	mape = 0
	if type == 'linear':
		alpha, beta = params
		a = [Y[0]]
		b = [Y[1] - Y[0]]
		y = [a[0] + b[0]]

		for i in range(len(Y)):
			a.append(alpha * Y[i] + (1 - alpha) * (a[i] + b[i]))
			b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
			y.append(a[i + 1] + b[i + 1])
	else:
		alpha, beta, gamma = params
		m = kwargs[2]
		a = [sum(Y[0:m]) / float(m)]
		b = [(sum(Y[m:2 * m]) - sum(Y[0:m])) / m ** 2]
        
		if type == 'additive':
			s = [Y[i] - a[0] for i in range(m)]
			y = [a[0] + b[0] + s[0]]
			for i in range(len(Y)):
				a.append(alpha * (Y[i] - s[i]) + (1 - alpha) * (a[i] + b[i]))
				b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
				s.append(gamma * (Y[i] - a[i] - b[i]) + (1 - gamma) * s[i])
				y.append(a[i + 1] + b[i + 1] + s[i + 1])

		elif type == 'multiplicative':
			s = [Y[i] / a[0] for i in range(m)]
			y = [(a[0] + b[0]) * s[0]]
			for i in range(len(Y)):
				a.append(alpha * (Y[i] / s[i]) + (1 - alpha) * (a[i] + b[i]))
				b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
				s.append(gamma * (Y[i] / (a[i] + b[i])) + (1 - gamma) * s[i])
				y.append((a[i + 1] + b[i + 1]) * s[i + 1])
		else:
			exit('Type must be either linear, additive or multiplicative')

	rmse = sqrt(sum([(m - n) ** 2 for m, n in zip(Y, y[:-1])]) / len(Y))
	return rmse

In [8]:
def linear_trend_no_seasonality(x, fc, alpha = None, beta = None):
	Y = list(x[:])
	if (alpha == None or beta == None):
		initial_values = array([0.3, 0.1])
		boundaries = [(0, 1), (0, 1)]
		type = 'linear'
		xmin = [0.0, 0.0]
		xmax = [1.0, 1.0]
		bounds = [(low, high) for low, high in zip(xmin, xmax)]
		minimizer_kwargs = {"method":"L-BFGS-B", "args" : (Y, type), "bounds" : bounds}
		parameters = basinhopping(MAPE, x0 = initial_values, minimizer_kwargs = minimizer_kwargs,niter = 50)
		alpha, beta = parameters.x[0], parameters.x[1]
        
	a = [Y[0]]
	b = [Y[1] - Y[0]]
	y = [a[0] + b[0]]
	mape = 0
	for i in range(len(Y) + fc):
		if i == len(Y):            
			Y.append(a[-1] + b[-1])
		a.append(alpha * Y[i] + (1 - alpha) * (a[i] + b[i]))
		b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
		y.append(a[i + 1] + b[i + 1])

	mape = sum([abs((m-n)/float(m)) for m, n in zip(Y, y[:-1])])*100/float(len(Y[:-fc]))
	return Y,alpha, beta, mape

def linear_trend_additive_seasonality(x, m, fc, alpha = None, beta = None, gamma = None):
	Y = list(x[:])
	if (alpha == None or beta == None or gamma == None):
		initial_values = array([0.3, 0.1, 0.1])
		boundaries = [(0, 1), (0, 1), (0, 1)]
		type = 'additive'    
		xmin = [0.0, 0.0, 0.0]
		xmax = [1.0, 1.0, 1.0]
		bounds = [(low, high) for low, high in zip(xmin, xmax)]
		minimizer_kwargs = {"method":"L-BFGS-B", "args" : (Y, type, m), "bounds" : bounds}
		parameters = basinhopping(MAPE, x0 = initial_values, minimizer_kwargs = minimizer_kwargs,niter = 10)
		alpha, beta, gamma = parameters.x[0], parameters.x[1], parameters.x[2]
        
	a = [sum(Y[0:m]) / float(m)]
	b = [(sum(Y[m:2 * m]) - sum(Y[0:m])) / m ** 2]
	s = [Y[i] - a[0] for i in range(m)]
	y = [a[0] + b[0] + s[0]]
	mape = 0

	for i in range(len(Y) + fc):
		if i == len(Y):
			Y.append(a[len(a)-1] + b[len(b)-1] + s[len(s)-m])
		a.append(alpha * (Y[i] - s[i]) + (1 - alpha) * (a[i] + b[i]))
		b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
		s.append(gamma * (Y[i] - a[i] - b[i]) + (1 - gamma) * s[i])
		y.append(a[i + 1] + b[i + 1] + s[i + 1])
        
	mape = sum([abs((m-n)/float(m)) for m, n in zip(Y, y[:-1])])*100/float(len(Y[:-fc]))
	return Y,alpha, beta, gamma, mape

def linear_trend_multiplicative_seasonality(x, m, fc, alpha = None, beta = None, gamma = None):
	Y = list(x[:])
	if (alpha == None or beta == None or gamma == None):
		initial_values = array([0.2, 0.05, 0.15])
		type = 'multiplicative'
		xmin = [0.0, 0.0, 0.0]
		xmax = [1.0, 1.0, 1.0]
		bounds = [(low, high) for low, high in zip(xmin, xmax)]
		minimizer_kwargs = {"method":"L-BFGS-B", "args" : (Y, type, m), "bounds" : bounds}
		parameters = basinhopping(MAPE, x0 = initial_values, minimizer_kwargs = minimizer_kwargs,niter = 50)
		alpha, beta, gamma = parameters.x[0], parameters.x[1], parameters.x[2]

	a = [sum(Y[0:m]) / float(m)]
	b = [(sum(Y[m:2 * m]) - sum(Y[0:m])) / m ** 2]
	s = [Y[i] / a[0] for i in range(m)]
	y = [(a[0] + b[0]) * s[0]]
	mape = 0

	for i in range(len(Y) + fc):
		if i == len(Y):
			Y.append((a[len(a)-1] + b[len(b)-1]) * s[len(s)-m])
		a.append(alpha * (Y[i] / s[i]) + (1 - alpha) * (a[i] + b[i]))
		b.append(beta * (a[i + 1] - a[i]) + (1 - beta) * b[i])
		s.append(gamma * (Y[i] / (a[i] + b[i])) + (1 - gamma) * s[i])
		y.append((a[i + 1] + b[i + 1]) * s[i + 1])

	mape = sum([abs((m-n)/float(m)) for m, n in zip(Y, y[:-1])])*100/float(len(Y[:-fc]))
	return Y,alpha, beta, gamma, mape

In [ ]:
# fc - forcaset period 
# m - seasonality
# y - input timeseries
# Y - forecasted time series of length "len(y)+fc" 
# aplha, beta, gamma - optimal alpha, beta, gamma found

y = pd.read_csv('')
plt.plot(y, 'bo-', color='orange')
plt.title('Time Series')

In [ ]:
Y,alpha, beta, mape = linear_trend_no_seasonality(y, m, fc)
Y,alpha, beta, gamma, mape = linear_trend_additive_seasonality(y, m, fc)
Y,alpha, beta, gamma, mape = linear_trend_multiplicative_seasonality(y, m, fc)